In [1]:
import numpy as np
import h5py

import e2c as e2c_util

# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1", "2", "3"

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from keras import backend as K
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras import losses


# GPU memory management
import tensorflow as tf

Using TensorFlow backend.


In [2]:
# tf.session specification
# TensorFlow wizardry
config = tf.ConfigProto()

# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True

# Only allow a total of half the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 0.75

# Create a session with the above options specified.
K.tensorflow_backend.set_session(tf.Session(config=config))

In [3]:
def reconstruction_loss(x, t_decoded):
    '''
    Reconstruction loss for the plain VAE
    '''
    v = 0.1
    # return K.mean(K.sum((K.batch_flatten(x) - K.batch_flatten(t_decoded)) ** 2 / (2*v) + 0.5*K.log(2*np.pi*v), axis=-1))
    return K.mean(K.sum((K.batch_flatten(x) - K.batch_flatten(t_decoded)) ** 2 / (2*v), axis=-1))
    # return K.sum((K.batch_flatten(x) - K.batch_flatten(t_decoded)) ** 2, axis=-1)


def l2_reg_loss(qm):
    # 0.5 * (torch.log(pv) - torch.log(qv) + qv / pv + (qm - pm).pow(2) / pv - 1)
    # -0.5 * K.sum(1 + t_log_var - K.square(t_mean) - K.exp(t_log_var), axis=-1)
#     kl = -0.5 * (1 - p_logv + q_logv - K.exp(q_logv) / K.exp(p_logv) - K.square(qm - pm) / K.exp(p_logv))
    l2_reg = 0.5*K.square(qm)
    return K.mean(K.sum(l2_reg, axis=-1))


def get_flux_loss(m, state, state_pred):
    '''
    @params:  state, state_pred shape (batch_size, 60, 60, 2)
              p, p_pred shape (batch_size, 60, 60, 1)
              m shape (batch_size, 60, 60, 1)
    
    @return:  loss_flux: scalar
    
    Only consider discrepancies in total flux, not in phases (saturation not used) 
    '''
    
    perm = K.exp(m)
    p = K.expand_dims(state[:, :, :, 1], -1)
    p_pred = K.expand_dims(state_pred[:, :, :, 1], -1)

    #print(K.int_shape(xxx))
    
    tran_x = 1./perm[:, 1:, ...] + 1./perm[:, :-1, ...]
    tran_y = 1./perm[:, :, 1:, ...] + 1./perm[:, :, :-1, ...]
    flux_x = (p[:, 1:, ...] - p[:, :-1, ...]) / tran_x
    flux_y = (p[:, :, 1:, :] - p[:, :, :-1, :]) / tran_y
    flux_x_pred = (p_pred[:, 1:, ...] - p_pred[:, :-1, ...]) / tran_x
    flux_y_pred = (p_pred[:, :, 1:, :] - p_pred[:, :, :-1, :]) / tran_y

    loss_x = K.sum(K.abs(K.batch_flatten(flux_x) - K.batch_flatten(flux_x_pred)), axis=-1)
    loss_y = K.sum(K.abs(K.batch_flatten(flux_y) - K.batch_flatten(flux_y_pred)), axis=-1)

    loss_flux = K.mean(loss_x + loss_y)
    return loss_flux

In [4]:
def get_binary_sat_loss(state, state_pred):
    
    sat_threshold = 0.105
    sat = K.expand_dims(state[:, :, :, 0], -1)
    sat_pred = K.expand_dims(state_pred[:, :, :, 0], -1)
    
    
    sat_bool = K.greater_equal(sat, sat_threshold) #will return boolean values
    sat_bin = K.cast(sat_bool, dtype=K.floatx()) #will convert bool to 0 and 1  
    
    sat_pred_bool = K.greater_equal(sat_pred, sat_threshold) #will return boolean values
    sat_pred_bin = K.cast(sat_pred_bool, dtype=K.floatx()) #will convert bool to 0 and 1  
    
#     binary_loss = K.sum(K.abs(K.batch_flatten(sat_bin) - K.batch_flatten(sat_pred_bin)), axis=-1)
    
    binary_loss = losses.binary_crossentropy(sat_bin, sat_pred_bin)
    return K.mean(binary_loss)

In [5]:
def create_e2c(latent_dim, u_dim, input_shape):
    '''
    Creates a E2C.

    Args:
        latent_dim: dimensionality of latent space
        return_kl_loss_op: whether to return the operation for
                           computing the KL divergence loss.

    Returns:
        The VAE model. If return_kl_loss_op is True, then the
        operation for computing the KL divergence loss is
        additionally returned.
    '''

    encoder_ = e2c_util.create_encoder(latent_dim, input_shape)
    decoder_ = e2c_util.create_decoder(latent_dim, input_shape)
    transition_ = e2c_util.create_trans(latent_dim, u_dim)

    return encoder_, decoder_, transition_

In [6]:
def get_well_bhp_loss(state, state_pred, prod_well_loc):
    '''
    @params: state: shape (batch_size, 60, 60, 2)
             state_pred: shape (batch_size, 60, 60, 2)
             prod_well_loc: shape (5, 2)
             
    p_true: shape (batch_size, 60, 60, 1)
    p_pred: shape (batch_size, 60, 60, 1)
    
    @return: bhp_loss: scalar
    '''
    
    p_true = K.expand_dims(state[:, :, :, 1], -1)
    p_pred = K.expand_dims(state_pred[:, :, :, 1], -1)
    
    print(K.int_shape(prod_well_loc))
#     print(K.int_shape(prod_well_loc[0, :, 0]))
#     print(K.int_shape(prod_well_loc[0, :, 1]))
    print(K.int_shape(prod_well_loc[:, 0]))
    print(K.int_shape(prod_well_loc[:, 1]))
    print(prod_well_loc.shape[0])
    bhp_loss = 0
    for i in range(prod_well_loc.shape[0]):
        bhp_loss += K.sum(K.abs(p_true[:, prod_well_loc[i, 1], prod_well_loc[i, 0], :] - p_pred[:, prod_well_loc[i, 1], prod_well_loc[i, 0], :]))
#     bhp_loss = K.sum(p_true[:, prod_well_loc[:, 0], prod_well_loc[:, 1], :] - p_pred[:, prod_well_loc[:,0], prod_well_loc[:,1], :])
    
    return bhp_loss

In [7]:
# Create plain E2C model and associated loss operations

################### case specification ######################

# data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_BHP/'
# data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_BHP_RATE/'
# data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_MS_BHP_RATE/'
data_dir = '/data3/Astro/personal/zjin/datasets/9W_MS_BHP_RATE_GAU/'
output_dir = '/data3/Astro/lstm_rom/e2c_larry/saved_models/'

#     case_name = '9w_bhp'
# case_name = '9w_bhp_rate'
case_name = '9w_ms_bhp_rate'

#     case_suffix = '_single_out_rel_2'
case_suffix = '_fix_wl_rel_8'
#     case_suffix = '_single_out_rel_3'
train_suffix = '_with_p'
model_suffix = '_flux_loss'

n_train_run = 100
n_eval_run = 100
num_t = 20
# dt = 200 // num_t
dt = 100
n_train_step = n_train_run * num_t
n_eval_step = n_eval_run * num_t


train_file = case_name + '_e2c_train' + case_suffix + train_suffix + '_n%d_dt%dday_nt%d_nrun%d.mat' %(n_train_step, dt, num_t, n_train_run)
eval_file = case_name + '_e2c_eval' + case_suffix + train_suffix +'_n%d_dt%dday_nt%d_nrun%d.mat' %(n_eval_step, dt, num_t, n_eval_run)

#################### model specification ##################
epoch = 10
batch_size = 4
learning_rate = 1e-4
latent_dim = 50
u_dim = 9*2 # control dimension

# load data
hf_r = h5py.File(data_dir + train_file, 'r')
state_t_train = np.array(hf_r.get('state_t'))
state_t1_train = np.array(hf_r.get('state_t1'))
bhp_train = np.array(hf_r.get('bhp'))
dt_train = np.array(hf_r.get('dt'))
hf_r.close()

num_train = state_t_train.shape[0]
# dt_train = np.ones((num_train,1)) # dt=20days, normalized to 1

hf_r = h5py.File(data_dir + eval_file, 'r')
state_t_eval = np.array(hf_r.get('state_t'))
state_t1_eval = np.array(hf_r.get('state_t1'))
bhp_eval = np.array(hf_r.get('bhp'))
dt_eval = np.array(hf_r.get('dt'))
hf_r.close()

num_eval = state_t_eval.shape[0]
# dt_eval = np.ones((num_eval, 1)) # dt=20days, normalized to 1

In [8]:
# m = np.loadtxt("/data/cees/zjin/lstm_rom/sim_runs/case6_9w_bhp_rate_ms_h5/template/logk1.dat")
m = np.loadtxt("/data3/Astro/personal/zjin/sim_runs/case6_9w_bhp_rate_ms_h5/template/logk1.dat")
m = m.reshape(60, 60, 1)
print('m shape is ', m.shape)
#     m_tf = K.placeholder((batch_size, 60, 60 ,1))
m_tf = Input(shape=(60, 60, 1))


m_eval = np.repeat(np.expand_dims(m, axis = 0), state_t_eval.shape[0], axis = 0)
print("m_eval shape is ", m_eval.shape)
m = np.repeat(np.expand_dims(m,axis = 0), state_t_train.shape[0], axis = 0)
print("m shape is ", m.shape)

m shape is  (60, 60, 1)
m_eval shape is  (2000, 60, 60, 1)
m shape is  (2000, 60, 60, 1)


In [9]:
well_loc_file = '/data3/Astro/personal/zjin/sim_runs/case8_9w_bhp_rate_ms_gau/template/well_loc00.dat'
well_loc = np.loadtxt(well_loc_file).astype(int)
num_prod = well_loc[0,0]
num_inj = well_loc[0,1]
num_well = num_prod+num_inj
print(num_inj, num_prod)

prod_loc = well_loc[1:num_prod+1,:]
print("prod_loc:\n{}".format(prod_loc))
# inj_loc = well_loc[num_prod+1:,:]
# print("inj_loc:\n{}".format(inj_loc))

print(prod_loc.shape)

print('prod_loc shape is ', prod_loc.shape)
# prod_loc_tf = Input(shape=(5, 2), dtype ='int32')
prod_loc_tf = tf.placeholder(tf.int32, shape=(5,2))

# prod_loc_eval = np.repeat(np.expand_dims(prod_loc, axis = 0), state_t_eval.shape[0], axis = 0)
# print("prod_loc_eval shape is ", prod_loc_eval.shape)
# prod_loc = np.repeat(np.expand_dims(prod_loc,axis = 0), state_t_train.shape[0], axis = 0)
# print("prod_loc shape is ", prod_loc.shape)

4 5
prod_loc:
[[10 10]
 [ 8 52]
 [30 30]
 [50 10]
 [50 50]]
(5, 2)
prod_loc shape is  (5, 2)


In [10]:
# Construct E2C
input_shape = (60, 60, 2)
encoder, decoder, transition = create_e2c(latent_dim, u_dim, input_shape)

xt = Input(shape=input_shape)
xt1 = Input(shape=input_shape)
ut = Input(shape=(u_dim, ))
dt = Input(shape=(1,))

zt = encoder(xt)
xt_rec = decoder(zt)

zt1 = encoder(xt1)

zt1_pred = transition([zt, ut, dt])
xt1_pred = decoder(zt1_pred)

# Compute loss
loss_rec_t = reconstruction_loss(xt, xt_rec)
loss_rec_t1 = reconstruction_loss(xt1, xt1_pred)

loss_flux_t = get_flux_loss(m_tf, xt, xt_rec) / 1000.
loss_flux_t1 = get_flux_loss(m_tf, xt1, xt1_pred) / 1000.

binary_sat_loss_t = get_binary_sat_loss(xt, xt_rec) * 1
binary_sat_loss_t1 = get_binary_sat_loss(xt1, xt1_pred) * 1

loss_prod_bhp_t = get_well_bhp_loss(xt, xt_rec, prod_loc_tf) * 20
loss_prod_bhp_t1 = get_well_bhp_loss(xt1, xt1_pred, prod_loc_tf) * 20

loss_l2_reg = l2_reg_loss(zt)  # log(1.) = 0.
# loss_bound = loss_rec_t + loss_rec_t1 + loss_l2_reg  + loss_flux_t + loss_flux_t1
# loss_bound = loss_rec_t + loss_rec_t1 + loss_kl + binary_sat_loss_t + binary_sat_loss_t1
loss_bound = loss_rec_t + loss_rec_t1 + loss_l2_reg  + loss_flux_t + loss_flux_t1 + loss_prod_bhp_t + loss_prod_bhp_t1




# Use zt_logvar to approximate zt1_logvar_pred
loss_trans = l2_reg_loss(zt1_pred - zt1)
# loss_trans = kl_normal_loss(zt1_mean_pred, zt1_logvar_pred, zt1_mean, zt1_logvar)


trans_loss_weight = 1.0 # lambda in E2C paper Eq. (11)
loss = loss_bound + trans_loss_weight * loss_trans

(5, 2)
(5,)
(5,)
5
(5, 2)
(5,)
(5,)
5


In [11]:
# Optimization
opt = Adam(lr=learning_rate)

trainable_weights = encoder.trainable_weights + decoder.trainable_weights + transition.trainable_weights

updates = opt.get_updates(loss, trainable_weights)

# iterate = K.function([xt, ut, xt1, m_tf, dt], [loss, loss_rec_t, loss_rec_t1, loss_l2_reg, loss_trans, loss_flux_t, loss_flux_t1, binary_sat_loss_t, binary_sat_loss_t1], updates=updates)
iterate = K.function([xt, ut, xt1, m_tf, prod_loc_tf, dt], [loss, loss_rec_t, loss_rec_t1, loss_l2_reg, loss_trans, loss_flux_t, loss_flux_t1, loss_prod_bhp_t, loss_prod_bhp_t1], updates=updates)

eval_loss = K.function([xt, ut, xt1, m_tf, prod_loc_tf, dt], [loss])

num_batch = int(num_train/batch_size)

for e in range(epoch):
    for ib in range(num_batch):
        ind0 = ib * batch_size
        state_t_batch = state_t_train[ind0:ind0+batch_size, ...]
        state_t1_batch = state_t1_train[ind0:ind0 + batch_size, ...]
        bhp_batch = bhp_train[ind0:ind0 + batch_size, ...]
        m_batch = m[ind0:ind0 + batch_size, ...]
        
        dt_batch = dt_train[ind0:ind0 + batch_size, ...]
#         print(dt_batch.shape)
        

        output = iterate([state_t_batch, bhp_batch, state_t1_batch, m_batch, prod_loc, dt_batch])

        # tf.session.run(feed_dict={xt: sat_t_batch, ut: bhp_batch, xt1: sat_t1_batch}, ...
        #                fetches= [loss, loss_rec_t, loss_rec_t1, loss_kl, loss_trans, updates])
        # But output tensor for the updates operation is not returned

        if ib % 10 == 0:
            print('Epoch %d/%d, Batch %d/%d, Loss %f, Loss rec %f, loss rec t1 %f, loss kl %f, loss_trans %f, loss flux %f, loss flux t1 %f, prod bhp loss %f, prod bhp loss t1 %f'
                  % (e+1, epoch, ib+1, num_batch, output[0], output[1], output[2], output[3], output[4], output[5], output[6], output[7], output[8]))
    eval_loss_val = eval_loss([state_t_eval, bhp_eval, state_t1_eval, m_eval, prod_loc, dt_eval])

    print('Epoch %d/%d, Train loss %f, Eval loss %f' % (e + 1, epoch, output[0], eval_loss_val[0]))


encoder.save_weights(output_dir + 'e2c_encoder_dt_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                     % (num_train, latent_dim, learning_rate, epoch))
decoder.save_weights(output_dir + 'e2c_decoder_dt_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                     % (num_train, latent_dim, learning_rate, epoch))
transition.save_weights(output_dir + 'e2c_transition_dt_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                        % (num_train, latent_dim, learning_rate, epoch))

Epoch 1/10, Batch 1/500, Loss 13105.566406, Loss rec 5799.131836, loss rec t1 6866.061523, loss kl 0.097017, loss_trans 0.123005, loss flux 69.627426, loss flux t1 90.025002, prod bhp loss 143.362366, prod bhp loss t1 137.138626
Epoch 1/10, Batch 11/500, Loss 2539.446289, Loss rec 503.807129, loss rec t1 1546.913574, loss kl 24.444771, loss_trans 31.745731, loss flux 218.031662, loss flux t1 132.415100, prod bhp loss 30.991003, prod bhp loss t1 51.097107
Epoch 1/10, Batch 21/500, Loss 1700.988892, Loss rec 305.131653, loss rec t1 919.912842, loss kl 23.803991, loss_trans 30.024172, loss flux 203.110138, loss flux t1 168.609955, prod bhp loss 26.623785, prod bhp loss t1 23.772505
Epoch 1/10, Batch 31/500, Loss 1481.031128, Loss rec 298.017609, loss rec t1 724.641357, loss kl 21.136948, loss_trans 25.008839, loss flux 190.958908, loss flux t1 175.042328, prod bhp loss 24.377037, prod bhp loss t1 21.848175
Epoch 1/10, Batch 41/500, Loss 951.030579, Loss rec 271.218811, loss rec t1 263.339

Epoch 1/10, Batch 371/500, Loss 436.923370, Loss rec 132.765411, loss rec t1 113.445465, loss kl 1.731745, loss_trans 1.175753, loss flux 68.607719, loss flux t1 66.078804, prod bhp loss 25.695538, prod bhp loss t1 27.422943
Epoch 1/10, Batch 381/500, Loss 649.854431, Loss rec 244.556213, loss rec t1 208.564087, loss kl 1.562813, loss_trans 0.876680, loss flux 60.926971, loss flux t1 57.343575, prod bhp loss 35.707603, prod bhp loss t1 40.316467
Epoch 1/10, Batch 391/500, Loss 521.230347, Loss rec 175.216980, loss rec t1 183.050079, loss kl 1.629509, loss_trans 1.001741, loss flux 60.076065, loss flux t1 58.938744, prod bhp loss 22.471310, prod bhp loss t1 18.845837
Epoch 1/10, Batch 401/500, Loss 1146.317627, Loss rec 403.249023, loss rec t1 568.192505, loss kl 2.419028, loss_trans 1.687499, loss flux 64.705208, loss flux t1 67.305313, prod bhp loss 20.355062, prod bhp loss t1 18.403973
Epoch 1/10, Batch 411/500, Loss 514.356873, Loss rec 159.425598, loss rec t1 204.179398, loss kl 1.

Epoch 2/10, Batch 241/500, Loss 1192.830933, Loss rec 380.532837, loss rec t1 579.480774, loss kl 0.394094, loss_trans 0.391632, loss flux 61.418858, loss flux t1 52.525230, prod bhp loss 52.602753, prod bhp loss t1 65.484756
Epoch 2/10, Batch 251/500, Loss 540.735901, Loss rec 173.848206, loss rec t1 200.581604, loss kl 0.832741, loss_trans 0.647641, loss flux 58.770130, loss flux t1 54.369038, prod bhp loss 23.659985, prod bhp loss t1 28.026520
Epoch 2/10, Batch 261/500, Loss 804.069153, Loss rec 297.096985, loss rec t1 351.600281, loss kl 0.387483, loss_trans 0.337040, loss flux 63.066948, loss flux t1 61.877033, prod bhp loss 18.131447, prod bhp loss t1 11.571918
Epoch 2/10, Batch 271/500, Loss 913.933533, Loss rec 307.899200, loss rec t1 385.616394, loss kl 0.394293, loss_trans 0.349301, loss flux 63.739197, loss flux t1 57.240623, prod bhp loss 43.968678, prod bhp loss t1 54.725845
Epoch 2/10, Batch 281/500, Loss 930.514832, Loss rec 322.827850, loss rec t1 377.688019, loss kl 0.

Epoch 3/10, Batch 111/500, Loss 842.191101, Loss rec 278.483459, loss rec t1 434.318542, loss kl 1.196126, loss_trans 0.856384, loss flux 51.538956, loss flux t1 48.842915, prod bhp loss 11.725519, prod bhp loss t1 15.229242
Epoch 3/10, Batch 121/500, Loss 396.271637, Loss rec 116.194824, loss rec t1 109.955307, loss kl 0.402313, loss_trans 0.395311, loss flux 57.623878, loss flux t1 51.124229, prod bhp loss 32.199802, prod bhp loss t1 28.375959
Epoch 3/10, Batch 131/500, Loss 403.108246, Loss rec 134.745667, loss rec t1 140.882904, loss kl 0.310932, loss_trans 0.266707, loss flux 53.909286, loss flux t1 51.476658, prod bhp loss 14.214762, prod bhp loss t1 7.301316
Epoch 3/10, Batch 141/500, Loss 290.788605, Loss rec 83.917252, loss rec t1 90.332153, loss kl 0.351738, loss_trans 0.300857, loss flux 47.637440, loss flux t1 44.825573, prod bhp loss 15.321419, prod bhp loss t1 8.102189
Epoch 3/10, Batch 151/500, Loss 375.532227, Loss rec 111.047897, loss rec t1 145.882645, loss kl 0.90269

Epoch 3/10, Batch 481/500, Loss 453.242035, Loss rec 112.627441, loss rec t1 198.197220, loss kl 0.234691, loss_trans 0.208406, loss flux 38.546333, loss flux t1 41.450745, prod bhp loss 29.719679, prod bhp loss t1 32.257584
Epoch 3/10, Batch 491/500, Loss 333.526154, Loss rec 118.097740, loss rec t1 112.716270, loss kl 0.142197, loss_trans 0.128319, loss flux 43.103378, loss flux t1 41.702049, prod bhp loss 9.323296, prod bhp loss t1 8.312903
Epoch 3/10, Train loss 258.734192, Eval loss 13802.379883
Epoch 4/10, Batch 1/500, Loss 330.421722, Loss rec 96.551430, loss rec t1 127.661560, loss kl 0.528038, loss_trans 0.354698, loss flux 42.178886, loss flux t1 45.336853, prod bhp loss 8.438347, prod bhp loss t1 9.371927
Epoch 4/10, Batch 11/500, Loss 546.431335, Loss rec 123.783867, loss rec t1 264.330750, loss kl 0.180167, loss_trans 0.236146, loss flux 39.399525, loss flux t1 42.123558, prod bhp loss 39.211388, prod bhp loss t1 37.165924
Epoch 4/10, Batch 21/500, Loss 272.419678, Loss re

...

Epoch 7/10, Batch 331/500, Loss 163.656662, Loss rec 22.644552, loss rec t1 54.232002, loss kl 0.827161, loss_trans 0.306053, loss flux 33.754051, loss flux t1 38.765259, prod bhp loss 4.955982, prod bhp loss t1 8.171609
Epoch 7/10, Batch 341/500, Loss 112.498871, Loss rec 21.672665, loss rec t1 25.208496, loss kl 0.475557, loss_trans 0.200603, loss flux 25.895613, loss flux t1 27.914173, prod bhp loss 4.868915, prod bhp loss t1 6.262856
Epoch 7/10, Batch 351/500, Loss 234.026855, Loss rec 21.199661, loss rec t1 122.660515, loss kl 0.301618, loss_trans 0.454568, loss flux 27.198622, loss flux t1 28.741856, prod bhp loss 18.576614, prod bhp loss t1 14.893397
Epoch 7/10, Batch 361/500, Loss 170.250671, Loss rec 32.493435, loss rec t1 63.127937, loss kl 0.320231, loss_trans 0.178347, loss flux 26.531813, loss flux t1 28.251833, prod bhp loss 11.833974, prod bhp loss t1 7.513114
Epoch 7/10, Batch 371/500, Loss 147.771942, Loss rec 34.617260, loss rec t1 32.392685, loss kl 0.699591, loss_tr

Epoch 8/10, Batch 201/500, Loss 242.250046, Loss rec 67.830124, loss rec t1 84.409546, loss kl 0.555694, loss_trans 0.227694, loss flux 27.955147, loss flux t1 33.948689, prod bhp loss 14.444995, prod bhp loss t1 12.878159
Epoch 8/10, Batch 211/500, Loss 95.240356, Loss rec 12.651871, loss rec t1 25.775078, loss kl 0.182010, loss_trans 0.142841, loss flux 20.083099, loss flux t1 24.573584, prod bhp loss 4.309732, prod bhp loss t1 7.522150
Epoch 8/10, Batch 221/500, Loss 192.411942, Loss rec 36.664787, loss rec t1 79.066360, loss kl 0.612309, loss_trans 0.201143, loss flux 26.531286, loss flux t1 32.621159, prod bhp loss 6.019938, prod bhp loss t1 10.694962
Epoch 8/10, Batch 231/500, Loss 111.511734, Loss rec 13.087167, loss rec t1 41.376839, loss kl 0.483694, loss_trans 0.159131, loss flux 19.512619, loss flux t1 24.281801, prod bhp loss 4.161749, prod bhp loss t1 8.448736
Epoch 8/10, Batch 241/500, Loss 465.893188, Loss rec 88.991608, loss rec t1 263.046692, loss kl 0.145937, loss_tra

Epoch 9/10, Batch 71/500, Loss 129.465088, Loss rec 27.483067, loss rec t1 42.561203, loss kl 0.249427, loss_trans 0.147940, loss flux 19.529915, loss flux t1 23.709625, prod bhp loss 6.211490, prod bhp loss t1 9.572437
Epoch 9/10, Batch 81/500, Loss 88.234787, Loss rec 9.475077, loss rec t1 24.685661, loss kl 0.345732, loss_trans 0.149348, loss flux 17.582401, loss flux t1 22.995657, prod bhp loss 5.209507, prod bhp loss t1 7.791402
Epoch 9/10, Batch 91/500, Loss 146.706436, Loss rec 18.975956, loss rec t1 68.568123, loss kl 0.681335, loss_trans 0.310037, loss flux 17.954832, loss flux t1 24.490047, prod bhp loss 4.689765, prod bhp loss t1 11.036331
Epoch 9/10, Batch 101/500, Loss 120.538536, Loss rec 14.425294, loss rec t1 47.345482, loss kl 0.326967, loss_trans 0.189449, loss flux 19.791553, loss flux t1 24.544552, prod bhp loss 5.352437, prod bhp loss t1 8.562801
Epoch 9/10, Batch 111/500, Loss 199.621262, Loss rec 18.887047, loss rec t1 99.931313, loss kl 1.676671, loss_trans 0.64

Epoch 9/10, Batch 441/500, Loss 92.645874, Loss rec 15.457978, loss rec t1 26.048120, loss kl 0.268999, loss_trans 0.147734, loss flux 17.303322, loss flux t1 21.061405, prod bhp loss 5.328369, prod bhp loss t1 7.029936
Epoch 9/10, Batch 451/500, Loss 188.471344, Loss rec 24.764885, loss rec t1 91.963341, loss kl 1.097702, loss_trans 0.323287, loss flux 21.781769, loss flux t1 27.722517, prod bhp loss 7.964897, prod bhp loss t1 12.852945
Epoch 9/10, Batch 461/500, Loss 91.032585, Loss rec 10.325260, loss rec t1 24.472271, loss kl 0.580473, loss_trans 0.156058, loss flux 18.905434, loss flux t1 24.546495, prod bhp loss 4.490030, prod bhp loss t1 7.556565
Epoch 9/10, Batch 471/500, Loss 163.953552, Loss rec 15.378002, loss rec t1 81.548401, loss kl 0.320040, loss_trans 0.248732, loss flux 19.566492, loss flux t1 25.463497, prod bhp loss 9.185258, prod bhp loss t1 12.243128
Epoch 9/10, Batch 481/500, Loss 117.284958, Loss rec 28.230160, loss rec t1 27.679293, loss kl 0.217702, loss_trans 

Epoch 10/10, Batch 311/500, Loss 179.748795, Loss rec 26.515776, loss rec t1 77.011627, loss kl 1.118136, loss_trans 0.427109, loss flux 25.605310, loss flux t1 31.682199, prod bhp loss 8.528753, prod bhp loss t1 8.859878
Epoch 10/10, Batch 321/500, Loss 244.225555, Loss rec 25.303402, loss rec t1 129.200317, loss kl 1.454313, loss_trans 0.630415, loss flux 27.114998, loss flux t1 36.332127, prod bhp loss 9.317310, prod bhp loss t1 14.872684
Epoch 10/10, Batch 331/500, Loss 123.599739, Loss rec 13.762994, loss rec t1 42.036572, loss kl 0.757737, loss_trans 0.252499, loss flux 25.802610, loss flux t1 31.018297, prod bhp loss 4.139680, prod bhp loss t1 5.829353
Epoch 10/10, Batch 341/500, Loss 113.822807, Loss rec 24.165882, loss rec t1 28.447731, loss kl 0.448084, loss_trans 0.199597, loss flux 22.801062, loss flux t1 24.748770, prod bhp loss 6.239534, prod bhp loss t1 6.772141
Epoch 10/10, Batch 351/500, Loss 172.682220, Loss rec 16.982513, loss rec t1 75.897644, loss kl 0.361755, loss